# Create Confusion Matrix
    1. Load dataset(csv)
    2. Split dataset to X_train,X_test,y_train,y_test
    3. Run classifier, by .fit(X_train,y_train)
    4. Impact on the results, by .predict(X_test)
    5. Import confusion_matrix and Create-> cm = confusion_matrix(y_test, y_pred)

In [1]:
import pandas as pd
import numpy as np
import pickle
import xlwt
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
df_file = pd.read_csv('../data/df_dropSub_less20_dropNaResult.csv',delimiter=",", skip_blank_lines = True, 
                 error_bad_lines=False)
df_file = df_file.drop('Unnamed: 0',axis=1)
df_file = df_file.fillna(0)
df_file = df_file.replace(['A', 'B+', 'B', 'C+', 'C' , 'D+' , 'D' , 'F' , 'W' , 'S' , 'S#' , 'U' , 'U#'], 
                     [8, 7, 7, 6 , 6, 5, 5, 4, 3, 2, 2, 1, 1])

In [3]:
df_file

,Unnamed: 0.1,3COURSEID,4RESULT,0STUDENTID,1ACADYEAR,2SEMESTER,AT316,AT326,BA291,CJ315,...,TA395,TH161,TU100,TU110,TU120,TU122,TU130,TU154,PROVINCEID,SCHOOLGPA
0,0,CS101,6,316644,2552,1,0,0,0,0,...,0,0,0,0,0,0,0,0,12,3.32
1,1,CS102,6,316644,2552,1,0,0,0,0,...,0,0,0,0,0,0,0,0,12,3.32
2,2,EL171,5,316644,2552,1,0,0,0,0,...,0,0,0,0,0,0,0,0,12,3.32
3,3,SC135,4,316644,2552,1,0,0,0,0,...,0,0,0,0,0,0,0,0,12,3.32
4,4,SC185,6,316644,2552,1,0,0,0,0,...,0,0,0,0,0,0,0,0,12,3.32
5,5,TH161,6,316644,2552,1,0,0,0,0,...,0,0,0,0,0,0,0,0,12,3.32
6,6,TU154,5,316644,2552,1,0,0,0,0,...,0,0,0,0,0,0,0,0,12,3.32
7,7,CS111,5,316644,2552,2,0,0,0,0,...,0,6,0,0,0,0,0,5,12,3.32
8,8,EL172,4,316644,2552,2,0,0,0,0,...,0,6,0,0,0,0,0,5,12,3.32
9,9,MA211,4,316644,2552,2,0,0,0,0,...,0,6,0,0,0,0,0,5,12,3.32


In [4]:
count_courseId = df_file["3COURSEID"].value_counts() 
more20 = count_courseId

headers=list(df_file.columns.values)
subjects = []
countSub = 0

In [5]:
count = 0
subjects.sort()
precision_rf={}
df_precision = more20.drop('CS231').copy()

list_allsub = df_file.columns[4:]
allSubject_df = pd.DataFrame(columns=[subjects],index=[list_allsub])
top10_df = pd.DataFrame(columns=[subjects])

In [6]:
headers=list(df_file.columns.values)
subjects = []
countSub = 0
#Create dictionary of list subjects
for sub in df_file[headers[1]]:
    if sub not in subjects:
        subjects.append(sub)
        countSub = countSub+1

In [7]:
subjects.sort()

In [8]:
subjects.remove('CS231')

In [9]:
len(subjects)

110

In [10]:
subject = 'CS401'
df_sub = df_file[df_file['3COURSEID'] == subject]
df_sub = df_sub.iloc[np.random.permutation(len(df_sub))]
count_enrollment = df_sub['3COURSEID'].value_counts()
#print "Number of %s enrollment: %s"%(subject,count_enrollment)

A = df_sub.as_matrix()
X = A[:,6:116]
X = X.astype(np.int64, copy=False)
y = A[:,2]
y = y.astype(np.int64, copy=False)

In [11]:
subject

'CS401'

In [12]:
X

array([[7, 8, 0, ..., 0, 6, 5],
       [7, 7, 0, ..., 0, 6, 7],
       [0, 0, 7, ..., 0, 6, 5],
       ..., 
       [0, 0, 0, ..., 0, 5, 4],
       [7, 7, 6, ..., 0, 6, 6],
       [8, 7, 0, ..., 5, 5, 3]], dtype=int64)

In [13]:
X.shape

(333L, 110L)

In [14]:
y.shape

(333L,)

In [15]:
# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=0)


In [16]:
X_train

array([[8, 8, 0, ..., 7, 6, 6],
       [0, 0, 0, ..., 0, 7, 6],
       [0, 0, 0, ..., 0, 6, 6],
       ..., 
       [0, 7, 0, ..., 5, 7, 6],
       [7, 7, 0, ..., 8, 6, 5],
       [7, 8, 0, ..., 7, 7, 5]], dtype=int64)

In [17]:
X.shape

(333L, 110L)

In [18]:
X_test

array([[0, 7, 0, ..., 7, 6, 5],
       [0, 0, 0, ..., 0, 6, 5],
       [0, 0, 0, ..., 0, 8, 8],
       ..., 
       [0, 0, 0, ..., 0, 6, 5],
       [7, 7, 0, ..., 4, 6, 7],
       [0, 0, 0, ..., 0, 5, 5]], dtype=int64)

In [19]:
X_test.shape

(67L, 110L)

In [20]:
X_train.shape

(266L, 110L)

In [21]:
y_train

array([8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 8, 8, 8, 7, 7, 8, 8, 7, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 7, 8, 7, 8, 7, 7, 8, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 6, 8, 8, 8, 7, 8, 8, 8, 8, 7, 8, 8, 8, 7, 8, 8, 7, 7, 8,
       8, 8, 8, 8, 8, 6, 8, 8, 8, 8, 8, 8, 8, 7, 8, 8, 8, 7, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 7, 8, 7, 8, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 7, 8, 8, 8, 8, 6, 8, 8, 8, 8, 8, 8, 8, 6, 7, 8, 6, 7, 8, 7, 8,
       8, 7, 8, 8, 8, 6, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 8, 8, 7,
       8, 8, 8, 8, 8, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 8, 8, 8, 7, 8, 8, 8,
       8, 8, 8, 7, 7, 8, 8, 8, 8, 8, 7, 8, 8, 7, 8, 7, 8, 8, 7, 8, 8, 8, 8,
       8, 8, 7, 8, 7, 7, 7, 6, 8, 7, 8, 8, 6, 8, 7, 8, 7, 8, 8, 8, 8, 7, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 8, 7, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 7, 8, 7, 8, 8, 8, 7, 8], dtype=int64)

In [22]:
y_train.shape

(266L,)

In [23]:
y_test

array([8, 7, 8, 8, 8, 8, 8, 8, 7, 8, 7, 7, 8, 7, 8, 8, 8, 8, 6, 7, 8, 8, 7,
       8, 8, 8, 8, 8, 8, 8, 8, 7, 8, 8, 8, 7, 8, 8, 8, 7, 7, 8, 7, 8, 8, 8,
       8, 8, 8, 7, 8, 8, 7, 7, 8, 8, 8, 8, 8, 8, 7, 8, 8, 7, 8, 8, 8], dtype=int64)

In [24]:
y_test.shape

(67L,)

In [25]:
forest = RandomForestClassifier(n_estimators=10, max_depth=None, 
            min_samples_split=1, random_state=None, max_features=None)
clf = forest.fit(X_train, y_train)

In [26]:
y_pred = clf.predict(X_test)

In [27]:
y_pred.shape

(67L,)

In [28]:
# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()

Confusion matrix, without normalization
[[ 0  0  1]
 [ 0  4 13]
 [ 0  3 46]]


In [29]:
y_pred

array([8, 8, 8, 7, 8, 8, 8, 8, 8, 8, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 7, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 8, 8, 8,
       8, 8, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8], dtype=int64)

In [30]:
y_test

array([8, 7, 8, 8, 8, 8, 8, 8, 7, 8, 7, 7, 8, 7, 8, 8, 8, 8, 6, 7, 8, 8, 7,
       8, 8, 8, 8, 8, 8, 8, 8, 7, 8, 8, 8, 7, 8, 8, 8, 7, 7, 8, 7, 8, 8, 8,
       8, 8, 8, 7, 8, 8, 7, 7, 8, 8, 8, 8, 8, 8, 7, 8, 8, 7, 8, 8, 8], dtype=int64)

In [31]:
Grade = ['A', 'B', 'C' , 'D' , 'F' , 'W' , 'S' , 'U' ,'na']
row = []
for cls in y_train:
    if cls not in row:
        row.append(cls)
row.sort()
print row

row_cm = []
for i in xrange(len(row)):
    Grade = ['A', 'B', 'C' , 'D' , 'F' , 'W' , 'S' , 'U' ,'na']
    grade = Grade[::-1][row[i]]
    print grade
    row_cm.append(grade)
print row_cm

[6, 7, 8]
C
B
A
['C', 'B', 'A']


In [32]:
row_cm

['C', 'B', 'A']

In [55]:
def plot_confusion_matrix(cm, title=subject, cmap=plt.cm.Blues):
   
    for y in range(cm.shape[0]):
        for x in range(cm.shape[1]):
            plt.text(x, y, '%.1f' % cm[y, x])
  
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(row_cm))
    plt.xticks(tick_marks, row_cm, rotation=45)
    plt.yticks(tick_marks, row_cm)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    
    plt.pcolor(cm, cmap=cmap)
    plt.yticks(np.arange(len(row_cm)),row_cm,label="#Training Data")
    plt.xticks(np.arange(len(row_cm)),row_cm,label='#Class')
    plt.colorbar()


In [56]:
# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
print('Confusion matrix')
print(row_cm)
print(cm)
plt.figure()
plot_confusion_matrix(cm)

Confusion matrix
['C', 'B', 'A']
[[ 0  0  1]
 [ 0  4 13]
 [ 0  3 46]]


In [57]:
cm

array([[ 0,  0,  1],
       [ 0,  4, 13],
       [ 0,  3, 46]])

In [58]:
plt.show()

# Confusion Matrix All Subjects
(has people enroll but no grade result)

In [46]:
for subject in subjects:
    #Create new Dataframe
    
    #print subject             
    df_sub = df_file[df_file['3COURSEID'] == subject]
    df_sub = df_sub.iloc[np.random.permutation(len(df_sub))]
    count_enrollment = df_sub['3COURSEID'].value_counts()
    #print "Number of %s enrollment: %s"%(subject,count_enrollment)

    A = df_sub.as_matrix()
    X = A[:,6:117]
    X = X.astype(np.int64, copy=False)
    y = A[:,2]
    y = y.astype(np.int64, copy=False)
    
    # Split the data into a training set and a test set
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=0)
    
    forest = RandomForestClassifier(n_estimators=10, max_depth=None, 
            min_samples_split=1, random_state=None, max_features=None)
    clf = forest.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    cm = confusion_matrix(y_test, y_pred)
    print('Confusion matrix of %s'%subject)
    print(cm)
    plt.figure()
    
    #plot_confusion_matrix(cm)
    

Confusion matrix of AT316
[[ 0  1  0]
 [ 0 14  1]
 [ 0 10  3]]
Confusion matrix of AT326
[[ 0  1  0]
 [ 0 16  8]
 [ 0 13  7]]
Confusion matrix of BA291
[[4 3 0]
 [3 1 0]
 [0 1 0]]
Confusion matrix of CJ315
[[0 0 0]
 [0 4 0]
 [1 1 0]]
Confusion matrix of CJ316
[[0 1 0]
 [0 4 2]
 [0 1 0]]
Confusion matrix of CJ317
[[4 2]
 [1 1]]
Confusion matrix of CJ321
[[7 0]
 [2 0]]
Confusion matrix of CS101
[[ 0  0  0  7  0  0]
 [ 0  0  1  5  1  0]
 [ 0  2  2 15  1  0]
 [ 0  0 11 85  6  0]
 [ 0  2  5 35  6  0]
 [ 0  1  0  5  0  0]]
Confusion matrix of CS102
[[ 0  0  0  2  1  0]
 [ 0  0  0  4  4  0]
 [ 0  0  4 32  5  0]
 [ 0  0 10 64 19  1]
 [ 0  0  4 21  9  0]
 [ 0  1  1  9  3  0]]
Confusion matrix of CS105
[[1 0 3 2 0 0]
 [1 0 0 1 0 0]
 [3 2 9 3 2 0]
 [1 1 7 4 1 0]
 [2 0 7 4 1 1]
 [0 0 2 0 2 0]]
Confusion matrix of CS111
[[ 2  4  4  2  0  0]
 [ 2  7  5  5  0  0]
 [ 1 13 17 18  1  0]
 [ 3  6 10 23  7  0]
 [ 0  1  2 10  5  4]
 [ 0  0  1  6  3  5]]
Confusion matrix of CS115
[[1 1 0]
 [1 0 0]
 [0 1 3]]


In [1]:
from sklearn.metrics import mean_squared_error

In [234]:
import pandas as pd
import numpy as np
from collections import defaultdict

df_file = pd.read_csv('../data/CS_table_No2_No4_new.csv',delimiter=";", skip_blank_lines = True, 
                 error_bad_lines=False)

In [235]:
for subject in subjects:
                 
    df_sub = df_file[df_file['3COURSEID'] == subject]
    df_sub = df_sub.iloc[np.random.permutation(len(df_sub))]
    count_enrollment = df_sub['3COURSEID'].value_counts()
    #print "Number of %s enrollment: %s"%(subject,count_enrollment)

    A = df_sub.as_matrix()
    X = A[:,6:116]
    X = X.astype(np.int64, copy=False)
    y = A[:,2]
    y = y.astype(np.int64, copy=False)

    #Training data
    forest = RandomForestClassifier(n_estimators=10, max_depth=None, 
            min_samples_split=1, random_state=None, max_features=None)
    clf = forest.fit(X, y)
    
    f = "tree_drop/tree%s.pic"%subject
    with open(f, 'wb') as pickleFile:
        pickle.dump(clf, pickleFile, pickle.HIGHEST_PROTOCOL)

,STUDENTID,ACADYEAR,SEMESTER,CAMPUSID,COURSEID,CAMPUSNAME,CURRIC,COURSENAME,SECTIONGROUP,CREDIT,GRADE
0,316644,2552,1,2,CS101,RANGSIT,521,DISCRETE STRUCTURES,10001,3,C
1,316644,2552,1,2,CS102,RANGSIT,521,COMPUTER PROGRAMMING FUNDAMENTALS,20301,4,C
2,316644,2552,1,2,EL171,RANGSIT,521,ENGLISH COURSE 2,100003,3,D
3,316644,2552,1,2,SC135,RANGSIT,521,GENERAL PHYSICS,80001,3,F
4,316644,2552,1,2,SC185,RANGSIT,521,GENERAL PHYSICS LABORATORY,5401,1,C
5,316644,2552,1,2,TH161,RANGSIT,521,THAI USAGE,90004,3,C
6,316644,2552,1,2,TU154,RANGSIT,521,FOUNDATION OF MATHEMATICS,70001,3,D
7,316644,2552,2,2,CS111,RANGSIT,521,OBJECT-ORIENTED PROGRAMMING,40501,4,D+
8,316644,2552,2,2,EL172,RANGSIT,521,ENGLISH COURSE 3,100003,3,F
9,316644,2552,2,2,MA211,RANGSIT,521,CALCULUS 1,90001,3,F
